# Black Scholes Model

In this notebook we illustrate the basic properties of the Black Scholes model.

The notebook is structured as follows:

  1. Black-Scholes model code
  2. Analysis of value function
  3. Analysis of Greeks, i.e. sensitivities to model parameters

## Black-Scholes Model Code

We use a couple of standard Python modules.

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import plotly.express as px
import plotly.graph_objects as go

As a basic building block we implement the Black formula.

$$
\begin{aligned}
    \text{Black}\left(F,K,\nu,\phi\right) &=\phi\,\left[F\,\Phi\left(\phi d_{1}\right)-K\,\Phi\left(\phi d_{2}\right)\right],\\
    d_{1,2}&=\frac{\log\left(F/K\right)}{\nu}\pm\frac{\nu}{2}.
\end{aligned}
$$

In [ ]:


def BlackOverK(moneyness, nu, callOrPut):
    d1 = np.log(moneyness) / nu + nu / 2.0
    d2 = d1 - nu
    return callOrPut * (moneyness*norm.cdf(callOrPut*d1)-norm.cdf(callOrPut*d2))

def Black(forward, strike, nu, callOrPut):
    if nu<1.0e-12:   # assume zero
        return np.maximum(callOrPut*(forward-strike),0.0)  # intrinsic value
    return strike * BlackOverK(forward/strike,nu,callOrPut)

def BlackImpliedVol(price, strike, forward, T, callOrPut):
    def objective(nu):
        return Black(forward, strike, nu, callOrPut) - price
    return brentq(objective,0.01*np.sqrt(T), 1.00*np.sqrt(T), xtol=1.0e-8) / np.sqrt(T)

def BlackVega(strike, forward, sigma, T):
    stdDev = sigma*np.sqrt(T)
    d1 = np.log(forward/strike) / stdDev + stdDev / 2.0    
    return forward * norm.pdf(d1) * np.sqrt(T)


## Analysis of Value Function

$$
  v(s,T) = e^{-rT}\,\text{Black}\left(s\,e^{rT},K,\sigma\sqrt{T},\phi\right),
$$

In [ ]:
def BlackScholesPrice(underlying, strike, rate, sigma, T, callOrPut):
    df = np.exp(-rate*T)
    nu = sigma*np.sqrt(T)
    return df * Black(underlying/df, strike, nu, callOrPut)

We need to specify some sensible model and product parameters.

In [ ]:
r     = 0.01  # 1% risk-free rate is a sensible choice in current low-interest rate market environment
sigma = 0.15  # typical values for annualised equity volatility is between 10% - 25%
K     = 1.0   # the strike should be in the order of the underlying asset; we will assume S~O(1)
phi   = 1.0   # call or put

We want to see the value function for a grid of maturities $[0,T_{end}]$ and underlying risky asset prices $(0, S_{max}]$.

In [ ]:
T = np.linspace(0.0, 2.0, 201)
S = np.linspace(0.01, 2.0, 200)

Now, we can calculate the call option prices.

In [ ]:
v = lambda s, t : BlackScholesPrice(s, K, r, sigma, t, phi)
v_sT = np.array([ v(S,t) for t in T ]).transpose()
print(v_sT.shape)

In [ ]:
fig = go.Figure(data=[go.Surface(x=T, y=S, z=v_sT)])
fig.update_layout(
    title='Black-Scholes Value Function',
    scene = dict(
        xaxis = dict(
            title = 'T',
        ),
        yaxis = dict(
            title = 's',
        ),
        zaxis = dict(
            title = 'v',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

## Analysis of Greeks

Greeks represent sensitivities of the value function with respect to changes in the model parameters.

### Delta

$$
  \Delta_{BS}(s,T)=\frac{d}{ds}v(s,T) = \phi\,\Phi\left(\phi d_{1}\right).
$$

In [ ]:
def BlackScholesDelta(underlying, strike, rate, sigma, T, callOrPut):
    moneyness = np.exp(rate*T) * underlying / strike
    nu = sigma * np.sqrt(T)
    d1 = np.log(moneyness) / nu + nu / 2.0
    return callOrPut * norm.cdf(callOrPut * d1)

We calculate the Delta for a range of underlyings and times.

In [ ]:
T = np.linspace(0.01, 2.0, 200)
S = np.linspace(0.01, 2.0, 200)
Delta = lambda s, t : BlackScholesDelta(s, K, r, sigma, t, phi)
dv_ds = np.array([ Delta(S,t) for t in T ]).transpose()
print(dv_ds.shape)


In [ ]:
# Check Delta via finite differences
eps = 1.0e-4
Delta_FD = lambda s, t : (BlackScholesPrice(s+eps, K, r, sigma, t, phi) - BlackScholesPrice(s-eps, K, r, sigma, t, phi))/2/eps
dv_ds_FD = np.array([ Delta_FD(S,t) for t in T ]).transpose()
print(np.max(np.abs(dv_ds-dv_ds_FD)))

And we plot the resulting sensitivity.

In [ ]:
fig = go.Figure(data=[go.Surface(x=T, y=S, z=dv_ds)])
fig.update_layout(
    title='Black-Scholes Delta',
    scene = dict(
        xaxis = dict(
            title = 'T',
        ),
        yaxis = dict(
            title = 's',
        ),
        zaxis = dict(
            title = 'Delta',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

### Gamma

$$
  \Gamma_{BS} = \frac{d}{ds}\Delta_{BS}(s,T)=\frac{d^{2}}{ds^{2}}v(s,T) = \frac{\Phi'\left(d_{1}\right)}{s\,\sigma\sqrt{T}}.
$$

In [ ]:
def BlackScholesGamma(underlying, strike, rate, sigma, T, callOrPut):
    moneyness = np.exp(rate*T) * underlying / strike
    nu = sigma * np.sqrt(T)
    d1 = np.log(moneyness) / nu + nu / 2.0
    return norm.pdf(d1) / underlying / nu

We calculate the Gamma for a range of underlyings and times.

In [ ]:
T = np.linspace(0.1, 2.0, 200)
S = np.linspace(0.01, 2.0, 200)
Gamma = lambda s, t : BlackScholesGamma(s, K, r, sigma, t, phi)
d2v_ds2 = np.array([ Gamma(S,t) for t in T ]).transpose()
print(d2v_ds2.shape)


In [ ]:
# Check Gamma via finite differences
eps = 1.0e-4
Gamma_FD = lambda s, t : (BlackScholesPrice(s+eps, K, r, sigma, t, phi) - 2 * BlackScholesPrice(s, K, r, sigma, t, phi) + BlackScholesPrice(s-eps, K, r, sigma, t, phi))/eps**2
d2v_ds2_FD = np.array([ Gamma_FD(S,t) for t in T ]).transpose()
print(np.max(np.abs(d2v_ds2 - d2v_ds2_FD)))

In [ ]:
fig = go.Figure(data=[go.Surface(x=T, y=S, z=d2v_ds2)])
fig.update_layout(
    title='Black-Scholes Gamma',
    scene = dict(
        xaxis = dict(
            title = 'T',
        ),
        yaxis = dict(
            title = 's',
        ),
        zaxis = dict(
            title = 'Gamma',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

### Theta

$$
  \Theta_{BS}(s,T)=\frac{d}{dT}v(s,T) = \frac{s\,\Phi'\left(d_{1}\right)\,\sigma}{2\,\sqrt{T}}+\phi\,r\,K\,e^{-rT}\,\Phi\left(\phi d_{2}\right)
$$

In [ ]:
def BlackScholesTheta(underlying, strike, rate, sigma, T, callOrPut):
    moneyness = np.exp(rate*T) * underlying / strike
    nu = sigma * np.sqrt(T)
    d1 = np.log(moneyness) / nu + nu / 2.0
    d2 = d1 - nu
    return underlying * norm.pdf(d1) * sigma / 2 / np.sqrt(T) + \
        callOrPut * rate * strike * np.exp(-rate*T) * norm.cdf(callOrPut * d2)

We calculate the Theta for a range of underlyings and times.

In [ ]:
T = np.linspace(0.1, 2.0, 200)
S = np.linspace(0.01, 2.0, 200)
Theta = lambda s, t : BlackScholesTheta(s, K, r, sigma, t, phi)
dv_dT = np.array([ Theta(S,t) for t in T ]).transpose()
print(dv_dT.shape)

In [ ]:
# Check Theta via finite differences
eps = 1.0e-4
Theta_FD = lambda s, t : (BlackScholesPrice(s, K, r, sigma, t+eps, phi) - BlackScholesPrice(s, K, r, sigma, t-eps, phi))/2/eps
dv_dT_FD = np.array([ Theta_FD(S,t) for t in T ]).transpose()
print(np.max(np.abs(dv_dT - dv_dT_FD)))

In [ ]:
fig = go.Figure(data=[go.Surface(x=T, y=S, z=dv_dT)])
fig.update_layout(
    title='Black-Scholes Theta',
    scene = dict(
        xaxis = dict(
            title = 'T',
        ),
        yaxis = dict(
            title = 's',
        ),
        zaxis = dict(
            title = 'Theta',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

### Black-Scholes PDE

We calculate the linear operator
$$
  {\cal L}\left[v\right]=-\frac{dv}{dT}+r\,s\,\frac{dv}{ds}+\frac{1}{2}\,\sigma^{2}\,s^{2}\,\frac{d^{2}v}{ds^{2}}-r\,v.
$$
And verify that ${\cal L}\left[v\right]=0$.

In [ ]:
T = np.linspace(0.1, 2.0, 200)
S = np.linspace(0.01, 2.0, 200)
L_v = lambda s, T : -Theta(s,T) + r * s * Delta(s,T) + 0.5 * sigma**2 * s**2 * Gamma(s,T) - r * v(s,T)
L_v_sT = np.array([ L_v(S,t) for t in T ]).transpose()
print(L_v_sT.shape)

In [ ]:
fig = go.Figure(data=[go.Surface(x=T, y=S, z=L_v_sT)])
fig.update_layout(
    title='Black-Scholes Operator',
    scene = dict(
        xaxis = dict(
            title = 'T',
        ),
        yaxis = dict(
            title = 's',
        ),
        zaxis = dict(
            title = 'L[v]',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

### Rho

$$
  \varrho_{BS}(s,T)=\frac{d}{dr}v(s,T) = \phi\,K\,T\,e^{-rT}\,\Phi\left(\phi d_{2}\right).
$$

In [ ]:
def BlackScholesRho(underlying, strike, rate, sigma, T, callOrPut):
    moneyness = np.exp(rate*T) * underlying / strike
    nu = sigma * np.sqrt(T)
    d1 = np.log(moneyness) / nu + nu / 2.0
    d2 = d1 - nu
    return callOrPut * strike * T * np.exp(-rate*T) * norm.cdf(callOrPut * d2)

We calculate the Theta for a range of underlyings and times.

In [ ]:
T = np.linspace(0.01, 2.0, 200)
S = np.linspace(0.01, 2.0, 200)
Rho = lambda s, t : BlackScholesRho(s, K, r, sigma, t, phi)
dv_dr = np.array([ Rho(S,t) for t in T ]).transpose()
print(dv_dr.shape)

In [ ]:
# Check Rho via finite differences
eps = 1.0e-6
Rho_FD = lambda s, t : (BlackScholesPrice(s, K, r+eps, sigma, t, phi) - BlackScholesPrice(s, K, r-eps, sigma, t, phi))/2/eps
dv_dr_FD = np.array([ Rho_FD(S,t) for t in T ]).transpose()
print(np.max(np.abs(dv_dr - dv_dr_FD)))

In [ ]:
fig = go.Figure(data=[go.Surface(x=T, y=S, z=dv_dr)])
fig.update_layout(
    title='Black-Scholes Rho',
    scene = dict(
        xaxis = dict(
            title = 'T',
        ),
        yaxis = dict(
            title = 's',
        ),
        zaxis = dict(
            title = 'Rho',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

### Vega

$$
  \text{Vega}_{BS}(s,T)=\frac{d}{d\sigma}v(s,T) = s\,\Phi'\left(d_{1}\right)\sqrt{T}
$$

In [ ]:
def BlackScholesVega(underlying, strike, rate, sigma, T, callOrPut):
    moneyness = np.exp(rate*T) * underlying / strike
    nu = sigma * np.sqrt(T)
    d1 = np.log(moneyness) / nu + nu / 2.0
    return underlying * norm.pdf(d1) * np.sqrt(T)

We calculate the Theta for a range of underlyings and times.

In [ ]:
T = np.linspace(0.01, 2.0, 200)
S = np.linspace(0.01, 2.0, 200)
Vega = lambda s, t : BlackScholesVega(s, K, r, sigma, t, phi)
dv_dsigma = np.array([ Vega(S,t) for t in T ]).transpose()
print(dv_dr.shape)

In [ ]:
# Check Vega via finite differences
eps = 1.0e-6
Vega_FD = lambda s, t : (BlackScholesPrice(s, K, r, sigma+eps, t, phi) - BlackScholesPrice(s, K, r, sigma-eps, t, phi))/2/eps
dv_dsigma_FD = np.array([ Vega_FD(S,t) for t in T ]).transpose()
print(np.max(np.abs(dv_dsigma - dv_dsigma_FD)))

In [ ]:
fig = go.Figure(data=[go.Surface(x=T, y=S, z=dv_dsigma)])
fig.update_layout(
    title='Black-Scholes Vega',
    scene = dict(
        xaxis = dict(
            title = 'T',
        ),
        yaxis = dict(
            title = 's',
        ),
        zaxis = dict(
            title = 'Vega',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

# Implied Volatility Analysis

We add an analysis of market-implied volatilities.

In [ ]:
S0 = 1.0  # initial asset price
T  = 1.4

putStrikes  = [ 0.60,   0.70,   0.80,   0.90,   1.00   ]
putPrices   = [ 0.0642, 0.0943, 0.1310, 0.1761, 0.2286 ]

callStrikes = [ 1.00,   1.10,   1.20,   1.30,   1.40   ]
callPrices  = [ 0.2204, 0.1788, 0.1444, 0.1157, 0.0929 ]

We can use strike $K=1$ and put-call parity to calculate the implied risk-free rate $r$,
$$
  r = -\frac{\log\left(1+\pi_{BS}\left(C^{put}\right)-\pi_{BS}\left(C^{call}\right)\right)}{T}
$$


In [ ]:
r = - np.log(1 + putPrices[-1] - callPrices[0])/T
r

Next, we can calculate implied volatilities for puts and calls.

In [ ]:
F = np.exp(r*T) * S0

putFwdPrices  = [ np.exp(r*T)*p for p in putPrices  ]
callFwdPrices = [ np.exp(r*T)*p for p in callPrices ]

putVols  = [ BlackImpliedVol(p,K,F,T,-1) for p, K in zip(putFwdPrices, putStrikes)  ]
callVols = [ BlackImpliedVol(p,K,F,T,+1) for p, K in zip(callFwdPrices,callStrikes) ]

print(putVols[-1])
print(callVols[0])

In [ ]:
sigma = 0.5 * (putVols[-1] + callVols[0])

We calculate the corresponding Black-Scholes model prices.

In [ ]:
bsPut  = [ BlackScholesPrice(S0,K,r,sigma,T,-1) for K in putStrikes  ]
bsCall = [ BlackScholesPrice(S0,K,r,sigma,T,+1) for K in callStrikes ]

print('Puts:')
for K, P in zip(putStrikes,bsPut):
    print('  %4.2f  %6.4f' % (K,P))
print('Calls:')
for K, P in zip(callStrikes,bsCall):
    print('  %4.2f  %6.4f' % (K,P))

Also, we plot the resulting impled volatility smile

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=putStrikes, y=putVols,   name='put' ))
fig.add_trace(go.Scatter(x=callStrikes, y=callVols, name='call'))
fig.update_layout(
    title='Implied Black-Scholes Volatility, T=%.2f' % T,
    xaxis_title="Strike K",
    yaxis_title="Implied Volatility",
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)

fig.show()